In [ ]:
#| default_exp experiments.merlion.metrics

In [ ]:
#| export
import sys
import logging
import numpy as np

from merlion.evaluate.forecast import ForecastScoreAccumulator as MerlionForecastScoreAccumulator

In [ ]:
#| export

logging.disable(sys.maxsize)

In [ ]:
#| export
def _mean_squared_error(
    *,
    ts: np.array,
    f: np.array,
) -> float:
    """
    Computes the mean squared error:

    $$(1/n)\sum^{n}_{i} | ts[i] - f[i]|**2, n = len(ts) = len(f)$$

    .Ignores nan values in times-series or the forecast.

    ------
    Parameters
    ------

    ts : np.array with the time-series
    f :  np.array with the forecast

    -------
    Returns
    -------
    
    """
    d = (ts - f)**2
    w = ~np.isnan(d)
    n = len(d[w])

    return np.divide(
        np.sum(d, where=w),
        n,
        where=n > 0,
    )

In [ ]:
#| export
def _relative_root_mean_square_error(
    *,
    ts: np.array,
    f: np.array,
) -> float:
    """
    Computes the root mean square error:

    $$ \sqrt((1/n)\sum^{n}_{i}(ts[i] - f[i])**2) / ((1/n) \sum^{n}_{i} ts[i] ) ,  n = len(ts) = len(f)$$

    .Ignores nan values in times-series or the forecast.

    ------
    Parameters
    ------

    ts : np.array with the time-series
    f :  np.array with the forecast

    -------
    Returns
    -------
    
    """

    mean = np.mean(ts)
    return np.divide(np.sqrt(_mean_squared_error(ts=ts, f=f)),mean, where=mean!=0)

In [ ]:
ts, f = np.array(
    [1, 2, 3],
    dtype=float,
), np.array(
    [.5, 2.5, 2],
    dtype=float,
)
assert _relative_root_mean_square_error(ts=ts, f=f) == np.sqrt(
    np.divide((1 - .5)**2 + (2 - 2.5)**2 + (3 - 2)**2, 3)) / np.mean(ts)

In [ ]:
#| export
def _root_mean_square_percentage_error(
    *,
    ts: np.array,
    f: np.array,
) -> float:
    """
    Computes the mean absolute percentage error:

    $$ \sqrt{ (1/n) \sum  ( (ts[i] - f[i])  /ts[i] ) ^ 2 },  n* = len(ts) = len(f)$$

    .Ignores nan values and division by zero.
    n is n* minus the ignored values

    ------
    Parameters
    ------

    ts : np.array with the time-series
    f :  np.array with the forecast

    -------
    Returns
    -------
    
    """

    pe = np.divide(ts - f, ts, where=ts != 0)[ts != 0]
    
    w = ~np.isnan(pe)

    n = len(pe[w])

    return np.sqrt(np.divide(
        np.sum(np.square(pe, where=w)),
        n,
        where=n > 0,
    ))

In [ ]:
ts, f = np.array([1, 1]), np.array([2,3])
assert _root_mean_square_percentage_error(ts=ts, f=f) == np.sqrt( np.divide( 2**2 +1, 2))

In [ ]:
ts, f = np.array([2, 1],dtype=float), np.array([3.,3],dtype=float)
assert _root_mean_square_percentage_error(ts=ts, f=f) == np.sqrt( np.divide( (1./2.)**2 +2**2, 2))

In [ ]:
ts, f = np.array([1, 2, 3]), np.array([.5, 2.5, 2])
assert _root_mean_square_percentage_error(ts=ts, f=f) == np.sqrt(np.divide(
    (1 - .5)**2 + ((2 - 2.5) / 2)**2 + ((3 - 2) / 3.)**2, 3))

In [ ]:
ts, f = np.array(
    [1, 2, 3, 0],
    dtype=float,
), np.array(
    [.5, 2.5, 2, 5],
    dtype=float,
)
assert _root_mean_square_percentage_error(ts=ts, f=f) == np.sqrt(
    np.divide((1 - .5)**2 + ((2 - 2.5) / 2)**2 + ((3 - 2) / 3.)**2, 3))

In [ ]:
#| export 
class ForecastScoreAccumulator(MerlionForecastScoreAccumulator):
    
    def rmspe(self):
        """
        Root Mean Square Percentage Error (RMSPE)

        For ground truth time series $ts$ and predicted time series $f$
        of length $n*$, it is computed as

        $$ \sqrt{ (1/n) \sum  ( (ts[i] - f[i])  / ts[i] ) ^ 2 } $$

        Ignores nan values and division by zero.
        n is n* minus the ignored values
        """
        self.check_before_eval()
        predict_values = self.predict.univariates[
            self.predict.names[0]].np_values
        ground_truth_values = self.ground_truth.univariates[
            self.ground_truth.names[0]].np_values
        

        return _root_mean_square_percentage_error(
            ts=ground_truth_values,
            f=predict_values,
        )
    
    def rrmse(self):
        """
        Relative Root Mean Square Error (RRMSE)

        For ground truth time series $ts$ and predicted time series $f$
        of length $n*$, it is computed as

        $$  RMSE / Mean $$
        
        where 
        $$ RMSE =  \sqrt((1/n)\sum^{n}_{i}(ts[i] - f[i])**2) $$ 
        and
        $$ ((1/n) \sum^{n}_{i} ts[i] ) $$
        

        Ignores nan values and division by zero.
        n is n* minus the ignored values
        """
        self.check_before_eval()
        predict_values = self.predict.univariates[
            self.predict.names[0]].np_values
        ground_truth_values = self.ground_truth.univariates[
            self.ground_truth.names[0]].np_values
        

        return _relative_root_mean_square_error(
            ts=ground_truth_values,
            f=predict_values,
        )

In [ ]:
from national.experiments.merlion.inference import Inference as MerlionInference

In [ ]:
mi = MerlionInference(
    kpi="Sales",
    freq="W",
)

In [ ]:

fsa = ForecastScoreAccumulator(
    ground_truth=mi.data.sub_test,
    predict=mi.models.arima.forecast.inference,
)

In [ ]:
fsa.rmspe()

0.15654322483622773

In [ ]:
fsa.rrmse()

0.1607804345030961